## Brief Introduction to Neural Networks



### Components of a neural network

![](GenericNeuralNetwork.jpg "Generic Neural Network")


### General details
Neural networks consist of an input layer, an output layer, and one or more hidden layers. Each layer is *connected* in some manner. This is a schematic of a *fully connected* network because each of the inputs is connected to *all* of the nodes, or neurons, in the following layer. This is sometimes called a *densely* connected network. 

This is an example of a *feed-forward* network in which the connections (links from one layer to another) flow in only one direction: from input to output. Each connection in a neural network is associated with a weight that must be learned through training. 

### Input layer
The input layer of this network consists of three input nodes, which are *features* or variables of our input data. No computation occurs on the input layer and these data are passed directly to the first hidden layer.

### Hidden layer
This network consists of two hidden layers, each with four nodes, which are typically called neurons in the hidden layers. 

The number of hidden layers and the number of neurons of each layer are design parameters of the network, or *hyperparameters*, and are components that one seeks to tune during optimization of a network for a given data set or application. 

Each neuron of a hidden layer computes an *activation* based on the input to the node. The input consists of the data being fed into it with an associated weight. 

The activation is determined by some activation function, which takes a single number and performs a fixed mathematical operation on it. The purpose of these functions is to introduce non-linearity into the output of the neuron, thus allowing neural networks to learn nonlinear relationships between the input and output. The activation also may allow the output of the neuron to take on a continuum of values rather than binary 0 or 1, which might lead to small changes in input producing large changes in output across the network. 

A few useful activation functions that are commonly used: 

###### Sigmoid
Takes real-valued input and squashes to range [0,1]
$$\sigma(x) = \frac{1}{1+e^{-1}}$$

![](SigmoidFunction.gif "")

###### Tanh
Takes real-valued input and squashes to range [-1,1]
$$\tanh(x) = 2\sigma(2x) − 1$$
*NOTE: Tanh is simply a scaled sigmoid function*

![](TanhFunction.gif "")


###### ReLU (Rectified Linear Unit)
Takes real-valued input and thresholds at zero (negative values become zero)
$$f(x) = max[0,x]$$

![](ReLUFunction.png "")



### Output layer
The output layer of this network has only one node, which performs a similar computation as the hidden layer neurons and calculates the output of the network. The output may be real-valued (regression), multiple discrete values (multinomial classification), or binary (binomial classification). The network may also produce more than one output (i.e., multiple variables are predicted).

The output ultimately represents whatever the *target* of the network is. The values of the target in a training data set are referred to as *labels*. 

### Neural networks can take many forms...  
![](NeuralNetworkTypes.jpg "Neural Network Types")


### High-level neural network operations
*NOTE: we will not get into the mathematical details of calculating and updating the weights within neural networks (e.g., backpropagation) today, but rather attempt to develop an intuition for neural network operation. A good initial resource is Andrew Ng's machine learning online Coursera course: https://bit.ly/2zvKm8C


#### How a neural network 'learns': Backpropagation
Backpropagation is a *supervised* technique, meaning that the model learns from labeled data (i.e., it knows the answer). 

All network weights are initially assigned random values. For every input data sample (a single instance of the input data) the weights of the network are activated and the output computed. This output is compared with the known expected output (the label) and the weights are adjusted accordingly (based, essentially on a simple chain rule for nested functions). The process repeats until the output error falls below a predefined threshold. 

The final result is a *learned* neural network and can then be used to predict the labels for unseen/new input data samples. 

Further information can be found at this excellent visual representation of backpropagation: https://bit.ly/2bF6NcM




### Example

Following https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/

### Tools 


###### Tensorflow
Tensorflow is an open source software library for machine intelligence originally developed by researchers and engineers working on the Google Brain Team

###### Keras
Keras is a high-level neural networks applied programming interface (API, a software intermediary that allows two applications to talk to each other) capable of running on top of Tensorflow and Theano backends
   - Very useful for rapid prototyping, quickly going from idea to result

###### Other packages
- Numpy
- Scipy
- Pandas


In [23]:
import keras, tensorflow

print( keras.__version__)
print( tensorflow.__version__)



2.2.2
1.10.1


##### Step 1: Load the data

Using the Pima indians onset of diabetes dataset, a standard dataset provided by UCI Machine Learning repository - https://archive.ics.uci.edu/ml/datasets.html
- It describes patient medical record data for Pima Indians and whether they had an onset of diabetes within five years
- All input data are numerical
- Output is a 0 (no onset diabetes) or 1 (onset diabetes)
- Requires that we download and save the data


In [26]:
from keras.models import Sequential
from keras.layers import Dense
import numpy as np
import pandas as pd

# fix random seed for reproducibility
np.random.seed(7)


dataset = pd.read_csv("pima_indians_data.csv",header=None)
# split into input (X) and output (Y) variables
X = dataset.iloc[:,0:8].values
Y = dataset.iloc[:,8].values


In [29]:
dataset.describe()

,0,1,2,3,4,5,6,7,8
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


##### Step 2: Create the neural network
- Input layer must match the number of input variables (8 in this example)
- The number and size of the layers is a critical design choice of the neural network, but we will simply choose them here
- Use 'Dense' property to create fully connected layers
- 'activation' property determines the activation function that we will use
   - using ReLU on the first two layers and sigmoid on the output layer, ensuring that the output is between 0 and 1 and easy to map to the binary classification problem
- Initialize the weights of the network using the 'uniform' distribution

In [30]:
input_dimension = X.shape[1]
activation_input = 'relu'
activation_hidden = 'relu'
activation_output = 'sigmoid'

model = Sequential()
model.add(Dense(12, input_dim=input_dimension, activation=activation_input))
model.add(Dense(8, activation=activation_hidden))
model.add(Dense(1, activation=activation_output))

##### Step 3: Compile the model
- 'Wrapped' libraries (here, Tensorflow) are called and details are determined for computing system and defined model
- Must define a few additional parameters
   - loss function to evaluate the set of weights of the current model
   - optimization strategy
   - metrics to define model performance

In [31]:
loss_function = 'binary_crossentropy'
optimization_strategy = 'adam'
evaluation_metric = 'accuracy'

model.compile(loss=loss_function, optimizer=optimization_strategy, metrics=[evaluation_metric])

##### Step 4: Fit the model
- Iterations through the training data defined by 'epochs'
- Number of data samples examined prior to updating weights defined by 'batch_size'


In [32]:
num_samples_batch = 10
num_epochs = 150
print('CPU working on fitting the model........')
model.fit(X, Y, epochs=num_epochs, batch_size=num_samples_batch)
print('.........CPU fit the model')




CPU working on fitting the model........
Epoch 1/150
768/768 [==============================] - 0s 580us/step - loss: 3.7114 - acc: 0.5977
Epoch 2/150
768/768 [==============================] - 0s 99us/step - loss: 0.9375 - acc: 0.5911
Epoch 3/150
768/768 [==============================] - 0s 97us/step - loss: 0.7475 - acc: 0.6419
Epoch 4/150
768/768 [==============================] - 0s 96us/step - loss: 0.7123 - acc: 0.6549

##### Step 5: Evaluate the model
- We will evaluate on the same data that was used to train the model, which is, in general, a poor indicator
- We will only be able to claim that the network has well modeled the data, NOT that it is capable of generalization
- More appropriate approach is to split the data into training and testing sets

In [33]:
scores = model.evaluate(X, Y)
print('{0}: {1}%'.format(model.metrics_names[1], scores[1]*100))

768/768 [==============================] - 0s 56us/step
acc: 78.125%


##### Step 6: Use the model to *predict*
- Could use 'model.predict(X)' to create predictions on unseen/unlabeled data

Fantastic reference on introduction to neural networks: http://cs231n.github.io/neural-networks-1/

...part of a course on neural networks for visual recognition offered in the Stanford Computer Science Department: http://cs231n.github.io/

### Play around!
- Much can be learned about the capability, complexity, and function of neural networks with tools that provide rapid feedback and prototyping (such as https://bit.ly/2OMKoOp)

## Recommendation: play around with neural networks on a data set that is relevant and interesting to you, not simply based on data availability/ease of use/etc.


![](HeliophysicsObsSystem.png "")


### References: 
- https://ujjwalkarn.me/2016/08/09/quick-intro-neural-networks/
- https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/
- UCI Machine Learning Dataset Repository: https://archive.ics.uci.edu/ml/datasets.html
- NASA Heliophysics Observational System Graphic: https://science.nasa.gov/heliophysics/programs/research